# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**⚠️ Make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating) before starting with this lab.** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [3]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [4]:
import spacy

def preprocess(text):
    # TODO: Replace the next line with your own code.
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    result = []
    for token in doc:
        if token.shape_ in ['Xxxxx', 'xxxx']:
            result.append(token.lemma_)
    return result

In [5]:
import spacy

def preprocess(text):
    # TODO: Replace the next line with your own code.
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    result = []
    for token in doc:
        if token.is_stop == False and token.is_alpha == True:
            result.append(token.lemma_)
    return result

In [6]:
# result is a list
# remove any stop words using token.is_stop == False;
# remove lemmas contain non-alphabeticla characters using token.is_alpha == True;
# add all the rest lemmas to result

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses the given text by tokenising it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

To speed up the preprocessing, you should disable loading those spaCy components you do not need, such as the parser, and named entity recogniser. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

### 🤞 Test your code

Test your implementation by running the following cell:

In [7]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

In [8]:
import spacy

def tokenizer(text):
    # TODO: Replace the next line with your own code.
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    result = []
    for token in doc:
        if token.is_stop == False and token.is_alpha == True:
            result.append(token.lemma_)
    return result

tokenizer('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

In [9]:
df['description']

0       ﻿10000000 is a Dungeon Crawling Puzzle RPG Mat...
1       I 1177 Vårdguidens app får du tillgång till 11...
2       Need counting games for kids & drawing for tod...
3       Beautiful and simple music application for tod...
4       A Fun and intuitive numbers game for your baby...
                              ...                        
1609    심심할 때 한번씩 즐기는 고스톱(맞고).\n복잡하고 요란한 화면이 부담스러우셨나요?...
1610    소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정...
1611    [2019년 8월 업데이트]\n- 신규 레전드 카드 추가\n- 메인 화면 이벤트 시...
1612    ★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국...
1613    Do you want to adopt a little cat but can’t do...
Name: description, Length: 1614, dtype: object

In [10]:
import numpy as np

# TODO: Replace the next line with your own code.
X = np.zeros((len(df), 1))
X

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df['description']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()


print(X.shape)



(1614, 27877)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df['description']
for corpu in corpus:
    tokenizer(corpu)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()


print(X.shape)


### 🤞 Test your code

Test your implementation by running the following cell:

In [ ]:
X.shape

This should show the dimensions of the matrix `X` to be around 1614 × 21389. (The number of rows should be exactly 1614. The number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.)

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [9]:
def search(query):
    # TODO: Replace the next line with your own code.
    return df.iloc[:10]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

### 🤞 Test your code

Test your implementation by running the following cell:

In [10]:
search('dodge trains')

,name,description
0,10000000,﻿10000000 is a Dungeon Crawling Puzzle RPG Mat...
1,1177 Vårdguiden,I 1177 Vårdguidens app får du tillgång till 11...
2,123 Draw🎨 Toddler counting for kids Drawing games,Need counting games for kids & drawing for tod...
3,123 Kids Fun MUSIC - Kids Music Educational Games,Beautiful and simple music application for tod...
4,123 number games for kids - Count & Tracing,A Fun and intuitive numbers game for your baby...
5,150 Animal Sounds,Animal Sounds is a very good entertainment gam...
6,1945,"The classic, long-run shooting game from the 9..."
7,2 3 4 Player Mini Games,Duel your friends in a variety of 4 player min...
8,2048,2048 is an addictive number puzzle game based ...
9,2D Strike,This is a fun shooter with a view from above. ...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [11]:
# TODO: Replace the next line with your own code.
terms = []

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [12]:
print(terms[:10])

[]


## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [13]:
def keywords(text, n=10):
    # TODO: Replace the next line with your own code.
    return []

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

### 🤞 Test your code

Test your implementation by running the following cell:

In [14]:
print(keywords(df['description'][1428]))

[]


This should give output similar to the following:

The exact output may differ slightly depending on the strategy used to break ties.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *application-specific* stop words?

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

*TODO: Enter your answers here*

**Congratulations on finishing L1! 👍**